In [1]:
import pandas as pd
import numpy as np
import os

# Prepare Demand

In [2]:
# Demand Size
demand_size_list = [i for i in range(100, 601, 50)]
# Demand Split Ratio (Intra Modal, %)
demand_split_ratio_list = [i for i in range(10, 91, 20)]

# Random Seed
random_seed_list = [3, 6, 9]

# Network Name
network_name = "11-500"

# Demand File Directory
demand_dir = f"data/demand/{network_name}/total/"

# Output Directory
output_dir = f"data/demand/{network_name}/amod/"

In [3]:
for rs in random_seed_list:
    for ds in demand_size_list:
        for dsr in demand_split_ratio_list:
            demand_name = f"ds{ds}_dsr{dsr}_rs{rs}.csv"
            demand_path = os.path.join(demand_dir, demand_name)
            if not os.path.exists(demand_path):
                print(f"{demand_name} does not exist. Skipping...")
                continue

            demand_df = pd.read_csv(demand_path)

            # Add FleetPy Required Columns and Default Values
            demand_df['is_multimodal'] = False
            demand_df['modal_state'] = 0  # 0: AMoD, 3: FLM
            demand_df['transfer_station_ids'] = None
            demand_df['max_transfers'] = 0

            # Get All Left Inter-Modal Trips
            left_inter_modal_trips = demand_df[(demand_df['rq_type'] == 'inter') & (demand_df['subnetwork'] == 'left')].copy()
            # Update Columns for Left Inter-Modal Trips
            left_inter_modal_trips['is_multimodal'] = True
            left_inter_modal_trips['modal_state'] = 3  # FLM
            left_inter_modal_trips['transfer_station_ids'] =  'A;B' 
            left_inter_modal_trips['max_transfers'] = 1

            # Get All Right Inter-Modal Trips
            right_inter_modal_trips = demand_df[(demand_df['rq_type'] == 'inter') & (demand_df['subnetwork'] == 'right')].copy()
            # Update Columns for Right Inter-Modal Trips
            right_inter_modal_trips['is_multimodal'] = True
            right_inter_modal_trips['modal_state'] = 3  # FLM
            right_inter_modal_trips['transfer_station_ids'] = 'B;A'
            right_inter_modal_trips['max_transfers'] = 1

            # Update Demand DataFrame
            demand_df.update(left_inter_modal_trips)
            demand_df.update(right_inter_modal_trips)

            # Save Updated Demand DataFrame
            output_path = os.path.join(output_dir, demand_name)
            os.makedirs(output_dir, exist_ok=True)
            demand_df.to_csv(output_path, index=False)

# Prepare FleetPy Configuration Files